Using https://www.kaggle.com/cyannani123/keras-cellular-image-classification as test example

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0,EfficientNetB1
from tensorflow.python.client import device_lib
import tensorflow.keras as keras
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input, concatenate, Convolution2D, MaxPool2D, Flatten
from tensorflow.keras.utils import Sequence
import os, sys, random,copy
import matplotlib.pyplot as plt

from PIL import Image
from PIL import ImageFilter
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
from tqdm import tqdm

In [3]:
sys.path.append('../src')

In [4]:
from data_v2 import augment
from data_v2 import get_center_box
from data_v2 import get_random_subbox
from data_v2 import ImgGen

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2021-11-25 21:57:16.504270: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-25 21:57:16.523224: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-25 21:57:16.523523: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.


Loading test and train data.

Will load directly from zip for now.

In [6]:
train_data_all = pd.read_csv("../data/train.csv")
print("Shape of train_data_all:", train_data_all.shape)

Shape of train_data_all: (36517, 5)


In [15]:
# get list of sirnas with most counts
train_data_all.loc[:,["id_code","sirna"]].groupby('sirna').count().to_csv("../data/train_sirna_counts.csv")

In [12]:
train_data_all=train_data_all.loc[train_data_all.sirna.isin(["sirna_706","sirna_1046","sirna_586","sirna_747"]),:]

In [13]:

sirna_label_encoder_all = LabelEncoder().fit(train_data_all.sirna)
print(f"Sirna classes in train_data_all {len(sirna_label_encoder_all.classes_)}")

Sirna classes in train_data_all 4


In [14]:

train_data_strat=train_data_all
#train_data_strat=train_data_all.groupby('sirna',group_keys=False).apply(lambda x: x.sample(frac=0.25).reset_index(drop=True))

train_data_sample_1 = train_data_strat.sample(frac=1,random_state=42).reset_index(drop=True)
print("Shape of train_data_sample_1:", train_data_sample_1.shape)
sirna_label_encoder_sample_1 = LabelEncoder().fit(train_data_sample_1.sirna)
print(f"Sirna classes in train_data_all {len(sirna_label_encoder_sample_1.classes_)}")


Shape of train_data_sample_1: (132, 5)
Sirna classes in train_data_all 4


Saidid on samast wellist tehtud eri pildid. Pm võib võtta ainult ühe saidi sisse.
Channelid on eri kanalitega tehtud pildid. Neid on kokku 6.

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input, concatenate, Convolution2D, MaxPool2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
from tensorflow.keras import layers

In [8]:
from tensorflow import keras

In [9]:
model = keras.Sequential(
    [
        keras.Input(shape=(6,224,224)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu", padding="same", input_shape=(6,224,224), data_format="channels_first"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(len(sirna_label_encoder_sample_1.classes_), activation="softmax"),
    ]
)

2021-11-25 20:41:14.220498: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-25 20:41:14.221582: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-25 20:41:14.221847: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-25 20:41:14.222055: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may hav

In [10]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 224, 224)      1760      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 112, 224)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 110, 64)       129088    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 55, 64)         0         
_________________________________________________________________
flatten (Flatten)            (None, 24640)             0         
_________________________________________________________________
dropout (Dropout)            (None, 24640)             0         
_________________________________________________________________
dense (Dense)                (None, 1108)              2

Training model

In [12]:
test_size = 0.05
batch_size = 32

In [13]:
train, val = train_test_split(train_data_sample_1, test_size=test_size)

In [14]:
print(f"Training set size {len(train)}")
print(f"Validation set size {len(val)}")

Training set size 17345
Validation set size 913


In [15]:
train_gen = ImgGen(train,batch_size=batch_size,preprocess=get_center_box,shuffle=True,label_encoder=sirna_label_encoder_sample_1, path='../data/train/')
val_gen = ImgGen(val,batch_size=batch_size,preprocess=get_center_box,shuffle=True,label_encoder=sirna_label_encoder_sample_1, path='../data/train/')

In [16]:
print(f"Training set batched size {len(train_gen)}")
print(f"Validation set batched size {len(val_gen)}")

Training set batched size 543
Validation set batched size 29


In [17]:
filepath = 'ModelCheckpoint_all.h5'

In [18]:
callback = [
        ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')
        ]

In [19]:
history = model.fit(train_gen, 
                              steps_per_epoch=len(train)//batch_size, 
                              epochs=100, 
                              verbose=1, 
                              validation_data=val_gen,
                              validation_steps=len(val)//batch_size,
                              callbacks=callback
                             )

2021-11-25 20:42:04.672702: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2021-11-25 20:42:06.100324: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201
2021-11-25 20:42:07.589880: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-11-25 20:42:07.606112: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-11-25 20:42:07.606151: W tensorflow/stream_executor/gpu/asm_compiler.cc:77] Couldn't get ptxas version string: Internal: Couldn't invoke ptxas --version
2021-11-25 20:42:07.622834: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-11-25 20:42:07.622912: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2021-11-25 20:42:09.118959: I tensorflow/stream_executor/cuda/c

144/542 [======>.......................] - ETA: 1:14 - loss: 7.0131 - accuracy: 2.1701e-04